# README.

This notebook shows how to do some very basic analysis of the measurements.

In [ ]:
!cp /whhdata/dbconnection.json .

In [ ]:
%matplotlib inline
import dbutils
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import date
from time import mktime
import calendar
import time
import numpy as np
from IPython.display import display

# Where to store outputs.
output_path = "."

# Connect to database.
main_connector = dbutils.connect_to_main_database()

measurement_types = ["manual", "v0.1", "v0.2"]

# Simple analytics of database data.

In [ ]:
main_connector = dbutils.connect_to_main_database()

sql_statement = "SELECT COUNT(*) FROM person;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "persons")

sql_statement = "SELECT COUNT(*) FROM measure;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "measures")

for measurement_type in measurement_types:
    sql_statement = "SELECT COUNT(*) FROM measure WHERE type='{}';".format(measurement_type)
    result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
    print(result, "measures with type {}".format(measurement_type))
    
sql_statement = "SELECT COUNT(*) FROM artifact;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE type='rgb';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts with type rgb")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts with type pcd")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE measure_id IS NOT NULL;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts with measure")

sql_statement = "SELECT COUNT(*) FROM artifact WHERE measure_id IS NULL;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifacts without measure")
    
sql_statement = "SELECT SUM(file_size) FROM artifact;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(int(result / (1024 * 1024 * 1024)), "GB artifacts data")

sql_statement = "SELECT SUM(file_size) FROM artifact WHERE type='rgb';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(int(result / (1024 * 1024 * 1024)), "GB artifacts data with type rgb")
      
sql_statement = "SELECT SUM(file_size) FROM artifact WHERE type='pcd';"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(int(result / (1024 * 1024 * 1024)), "GB artifacts data with type pcd")

sql_statement = "SELECT COUNT(DISTINCT(qr_code)) FROM artifact;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "unique QR codes on artifact level")

sql_statement = "SELECT COUNT(DISTINCT(qr_code)) FROM artifact WHERE measure_id IS NULL;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "unique QR codes on artifact level without measure_id")

sql_statement = "SELECT COUNT(*) FROM artifact_quality;"
result = main_connector.execute(sql_statement, fetch_all=True)[0][0]
print(result, "artifact quality KPIs")

# Getting statistical values of measurements.

In [ ]:
columns = [
 'age',
 'height',
 'weight',
 'muac',
 'head_circumference',
]

headers = []
rows = []
for column in columns:
    sql_statement = "SELECT MIN({0}), AVG({0}), MAX({0}) FROM measure WHERE type = 'manual' AND age > 0 AND age <10000;".format(column)
    minimum, average, maximum = main_connector.execute(sql_statement, fetch_one=True)
    
    rows.append([column, minimum, average, maximum])
    
headers = ("key", "minimum", "average", "maximum")
df = pd.DataFrame(rows, columns=headers)
df

# Creating pie charts for some columns.

In [ ]:
from collections import Counter

def make_pie(key):
    sql_statement = "SELECT {} FROM measure;".format(key)
    values_list = main_connector.execute(sql_statement, fetch_all=True)

    most_common = Counter(values_list).most_common(10000)

    # TODO percent
    values = []
    keys = []
    for key, value in most_common:
        print(key, value)
        keys.append(str(key))
        values.append(value)
        
    plt.figure(figsize=(15, 15))
    patches, texts = plt.pie(values)
    plt.legend(patches, keys)
    plt.title(df_key)
    plt.savefig(os.path.join(output_path, "pie-{}.png".format(df_key)))

    
df_keys = ["created_by", "oedema"]
for df_key in df_keys:
    make_pie(df_key)

# Render histograms.

In [ ]:
def render_histogram(key, values, bins, title):
    plt.figure(figsize=(20, 4))
    plt.hist(values, bins=bins)
    plt.title(title)
    plt.show()
    plt.close()
    
render_parameters = []
render_parameters.append(("age", 400, 3000))
render_parameters.append(("height", 400, 120))
render_parameters.append(("weight", 400, 30))
render_parameters.append(("muac", 400, 25))
render_parameters.append(("head_circumference", 400, 25))
for key, bins, maximum in render_parameters:
    sql_statement = "SELECT {} FROM measure WHERE type='manual' AND {}<{}".format(key, key, maximum)
    results = main_connector.execute(sql_statement, fetch_all=True)
    values = [result[0] for result in results]
    render_histogram(key, values, bins, "Distribution of {}".format(key))

# Render scatter plots.

In [ ]:
def render_scatterplot(key1, key2, x_values, y_values, title):
    plt.figure(figsize=(5, 5))
    plt.scatter(x_values, y_values, s=4)
    plt.xlabel(key1)
    plt.ylabel(key2)
    plt.title(title)
    plt.show()
    plt.close()
    
render_parameters = []
render_parameters.append(("height", "weight", 120, 20))
render_parameters.append(("height", "age", 120, 2000))
render_parameters.append(("weight", "age", 20, 2000))
render_parameters.append(("weight", "muac", 20, 20))
render_parameters.append(("weight", "head_circumference", 20, 200))
for key1, key2, maximum1, maximum2 in render_parameters:
    sql_statement = "SELECT {}, {} FROM measure WHERE type='manual'".format(key1, key2)
    sql_statement += " AND {}<{} AND {}<{}".format(key1, maximum1, key2, maximum2)
    results = main_connector.execute(sql_statement, fetch_all=True)
    results = np.array(results)
    x_values = results[:,0]
    y_values = results[:,1]
    render_scatterplot(key1, key2, x_values, y_values, "{} for {}".format(key1, key2))

# Specifying parameters.

In [ ]:
time_range = []
for month in range(6, 13):
    time_range.append((2018, month))
for month in range(1, 6):
    time_range.append((2018, month))
    
figsize=(10, 5)

# Render measures.

In [ ]:
for measurement_type in measurement_types:
    plot_values = []
    plot_labels = []
    for year, month in time_range:

        # Getting the start.
        start_day = 1
        start_date = date(year, month, start_day)
        start_timestamp = str(1000 * int(mktime(start_date.timetuple())))

        # Getting the end.
        _, end_day = calendar.monthrange(year, month)
        end_date = date(year, month, end_day)
        end_timestamp = str(1000 * int(mktime(end_date.timetuple())))

        # Getting the number of entries from the database.
        sql_statement = ""
        sql_statement += "SELECT COUNT(*) FROM measure"
        sql_statement += " WHERE type='{}'".format(measurement_type)
        sql_statement += " AND timestamp >= {}".format(start_timestamp)
        sql_statement += " AND timestamp <= {}".format(end_timestamp)
        number_of_entries = main_connector.execute(sql_statement, fetch_one=True)[0]

        # Prepare the plot.
        plot_values.append(number_of_entries)
        plot_labels.append("{}-{}".format(year, month))

    # Plot the plot.
    plt.figure(figsize=figsize)
    plt.bar(plot_labels, plot_values)
    plt.title("Number of measures over time with type {}".format(measurement_type))
    plt.show()
    plt.close()
    
    rows = []
    for row_date, row_count in zip(plot_labels, plot_values):
        rows.append((row_date, row_count))
    
    headers = ("Date", "Count")
    df = pd.DataFrame(rows, columns=headers)
    display(df)

# Artifacts and artifacts without measure.

In [ ]:
plot_values = []
plot_values_no_measure = []
plot_labels = []
for year, month in time_range:
        
    # Getting the start.
    start_day = 1
    start_date = date(year, month, start_day)
    start_timestamp = str(1000 * int(mktime(start_date.timetuple())))

    # Getting the end.
    _, end_day = calendar.monthrange(year, month)
    end_date = date(year, month, end_day)
    end_timestamp = str(1000 * int(mktime(end_date.timetuple())))

    # Getting the number of artifacts.
    sql_statement = ""
    sql_statement += "SELECT COUNT(*) FROM artifact"
    sql_statement += " WHERE create_date >= {}".format(start_timestamp)
    sql_statement += " AND create_date <= {}".format(end_timestamp)
    number_of_entries = main_connector.execute(sql_statement, fetch_one=True)[0]
    plot_values.append(number_of_entries)

    # Getting the number of artifacts without measures.
    sql_statement = ""
    sql_statement += "SELECT COUNT(*) FROM artifact"
    sql_statement += " WHERE create_date >= {}".format(start_timestamp)
    sql_statement += " AND create_date <= {}".format(end_timestamp)
    sql_statement += " AND measure_id IS NULL"
    number_of_entries = main_connector.execute(sql_statement, fetch_one=True)[0]
    plot_values_no_measure.append(number_of_entries)

    plot_labels.append("{}-{}".format(year, month))
        
# Plot the plot.
plt.figure(figsize=figsize)
plt.bar(plot_labels, plot_values, label="artifacts")
plt.bar(plot_labels, plot_values_no_measure, label="artifacts without measure")
plt.legend()
plt.title("Number of artifacts over time")
plt.show()
plt.close()

# Available data in GBs.

In [ ]:
plot_values_total = []
plot_values_images = []
plot_values_pointclouds = []
plot_labels = []
for year, month in time_range:
        
    # Getting the start.
    start_day = 1
    start_date = date(year, month, start_day)
    start_timestamp = str(1000 * int(mktime(start_date.timetuple())))

    # Getting the end.
    _, end_day = calendar.monthrange(year, month)
    end_date = date(year, month, end_day)
    end_timestamp = str(1000 * int(mktime(end_date.timetuple())))

    # Getting the overall filesize.
    sql_statement = ""
    sql_statement += "SELECT SUM(file_size) FROM artifact"
    sql_statement += " WHERE create_date >= {}".format(start_timestamp)
    sql_statement += " AND create_date <= {}".format(end_timestamp)
    number_of_entries = main_connector.execute(sql_statement, fetch_one=True)[0]
    if number_of_entries == None:
        number_of_entries = 0
    plot_values_total.append(number_of_entries)

    # Getting the overall filesize.
    sql_statement = ""
    sql_statement += "SELECT SUM(file_size) FROM artifact"
    sql_statement += " WHERE create_date >= {}".format(start_timestamp)
    sql_statement += " AND create_date <= {}".format(end_timestamp)
    sql_statement += " AND type='pcd'"
    number_of_entries = main_connector.execute(sql_statement, fetch_one=True)[0]
    if number_of_entries == None:
        number_of_entries = 0
    plot_values_pointclouds.append(number_of_entries)

    # Getting the overall filesize.
    sql_statement = ""
    sql_statement += "SELECT SUM(file_size) FROM artifact"
    sql_statement += " WHERE create_date >= {}".format(start_timestamp)
    sql_statement += " AND create_date <= {}".format(end_timestamp)
    sql_statement += " AND type='rgb'"
    number_of_entries = main_connector.execute(sql_statement, fetch_one=True)[0]
    if number_of_entries == None:
        number_of_entries = 0
    plot_values_images.append(number_of_entries)

    # Labels
    plot_labels.append("{}-{}".format(year, month))
        
# Plot the plot.
plt.figure(figsize=figsize)
plt.bar(plot_labels, plot_values_total, label="Total")
plt.bar(plot_labels, plot_values_images, label="Images")
plt.bar(plot_labels, plot_values_pointclouds, label="Pointclouds")
plt.legend()
plt.title("Artifact data in GBs")
plt.show()
plt.close()

# Number of QR codes on artifact level.

In [ ]:
plot_values = []
plot_labels = []
for year, month in time_range:
        
    # Getting the start.
    start_day = 1
    start_date = date(year, month, start_day)
    start_timestamp = str(1000 * int(mktime(start_date.timetuple())))

    # Getting the end.
    _, end_day = calendar.monthrange(year, month)
    end_date = date(year, month, end_day)
    end_timestamp = str(1000 * int(mktime(end_date.timetuple())))

    # Getting the number of entries from the database.
    sql_statement = ""
    sql_statement += "SELECT COUNT(DISTINCT(qr_code)) FROM artifact"
    sql_statement += " WHERE create_date >= {}".format(start_timestamp)
    sql_statement += " AND create_date <= {}".format(end_timestamp)
    number_of_entries = main_connector.execute(sql_statement, fetch_one=True)[0]

    # Prepare the plot.
    plot_values.append(number_of_entries)
    plot_labels.append("{}-{}".format(year, month))
        
# Plot the plot.
plt.figure(figsize=figsize)
plt.bar(plot_labels, plot_values)
plt.title("Number qr codes from artifacts.")
plt.show()
plt.close()

In [ ]:
plot_values = []
plot_labels = []
for year, month in time_range:
        
    # Getting the start.
    start_day = 1
    start_date = date(year, month, start_day)
    start_timestamp = str(1000 * int(mktime(start_date.timetuple())))

    # Getting the end.
    _, end_day = calendar.monthrange(year, month)
    end_date = date(year, month, end_day)
    end_timestamp = str(1000 * int(mktime(end_date.timetuple())))

    # Getting the number of entries from the database.
    sql_statement = ""
    sql_statement += "SELECT COUNT(*) FROM person"
    sql_statement += " WHERE timestamp >= {}".format(start_timestamp)
    sql_statement += " AND timestamp <= {}".format(end_timestamp)
    number_of_entries = main_connector.execute(sql_statement, fetch_one=True)[0]

    # Prepare the plot.
    plot_values.append(number_of_entries)
    plot_labels.append("{}-{}".format(year, month))
        
# Plot the plot.
plt.figure(figsize=figsize)
plt.bar(plot_labels, plot_values)
plt.legend()
plt.title("Number of persons.")
plt.show()
plt.close()

# Just some numbers.

In [ ]:
sql_statement = ""
sql_statement += "SELECT MIN(create_date), MAX(create_date) FROM artifact;"
min_create_date, max_create_date = main_connector.execute(sql_statement, fetch_one=True)

min_create_date = time.strftime("%a %d %b %Y %H:%M:%S GMT", time.gmtime(min_create_date / 1000.0))
print("First artifact:", min_create_date)

max_create_date = time.strftime("%a %d %b %Y %H:%M:%S GMT", time.gmtime(max_create_date / 1000.0))
print("Last artifact:", max_create_date)

In [ ]:
sql_statement = ""
sql_statement += "SELECT MIN(timestamp), MAX(timestamp) FROM measure;"
min_create_date, max_create_date = main_connector.execute(sql_statement, fetch_one=True)

min_create_date = time.strftime("%a %d %b %Y %H:%M:%S GMT", time.gmtime(min_create_date / 1000.0))
print("First measure:", min_create_date)

max_create_date = time.strftime("%a %d %b %Y %H:%M:%S GMT", time.gmtime(max_create_date / 1000.0))
print("Last measure:", max_create_date)